In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import gym

env = gym.make('Pendulum-v0')
print(env.observation_space)
print(env.action_space)

n_features = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]
b_actions = env.action_space.high[0]
n_hiddens = 30

print("n_features:", n_features, "n_actions:", n_actions, "b_actions:", b_actions)
print(torch.cuda.is_available())

Box(-8.0, 8.0, (3,), float32)
Box(-2.0, 2.0, (1,), float32)
n_features: 3 n_actions: 1 b_actions: 2.0
True


In [ ]:
class Actor(torch.nn.Module):
    def __init__(self, n_features, n_hiddens, n_actions, b_actions):
        super().__init__()
        self.n_features = n_features
        self.n_hiddens = n_hiddens
        self.n_actions = n_actions
        self.b_actions = b_actions

        self.fc1 = torch.nn.Linear(n_features, n_hiddens)
        self.fc2 = torch.nn.Linear(n_hiddens, n_actions)
        self.relu = torch.nn.ReLU()
        self.tanh = torch.nn.Tanh()

    def forward(self, x):
        assert len(x.shape) == 2
        assert x.shape[-1] == self.n_features

        x = self.relu(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = x * self.b_actions
        return x

class Critic(torch.nn.Module):
    def __init__(self, n_features, n_actions, n_hiddens):
        super().__init__()
        self.n_features = n_features
        self.n_actions = n_actions
        self.n_hiddens = n_hiddens

        self.fc_s = torch.nn.Linear(n_features, n_hiddens)
        self.fc_a = torch.nn.Linear(n_actions, n_hiddens)
        self.fc = torch.nn.Linear(n_hiddens, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, s, a):
        assert len(s.shape) == 2
        assert len(a.shape) == 2
        assert s.shape[-1] == self.n_features
        assert a.shape[-1] == self.n_actions
        assert s.shape[0] == a.shape[0]

        s = self.fc_s(s)
        a = self.fc_a(a)
        x = self.fc(self.relu(s + a))
        return x
    
class A3C:
    def __init__(self)